In [ ]:
!pip install sentence_transformers langchain PyPDF2 langchain


  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached langchain-0.0.266-py3-none-any.whl (1.5 MB)
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=36f3366623738ee7f3c8522b0115ee90e48

In [ ]:
!pip install InstructorEmbedding


In [ ]:
!pip install faiss-gpu faiss-cpu

In [ ]:
!pip install install llama-cpp-python

In [ ]:
import concurrent.futures
# from .instructor import *
import torch
from tqdm.autonotebook import trange
from PyPDF2 import PdfReader
import InstructorEmbedding
from transformers import T5Tokenizer
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
# from InstructorEmbedding import INSTRUCTOR
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

<ipython-input-11-c0dbeb5d9165>:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text, chunk_size=256):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

# def get_vectorstore_parallel(text_chunks, executor):
#     # Initialize HuggingFaceInstructEmbeddings
#     embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

#     # Initialize Hugging Face model and tokenizer
#     model_name = "hkunlp/instructor-xl"
#     tokenizer = T5Tokenizer.from_pretrained(model_name)

#     # Define the process_batch function
#     def process_batch(batch):
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
#         with torch.no_grad():
#             outputs = embeddings.encode(batch)  # Use the embeddings to encode the batch
#         return outputs

#     batch_size = 4
#     encoded_chunks = []
#     for i in range(0, len(text_chunks), batch_size):
#         batch = text_chunks[i:i + batch_size]
#         results = list(executor.map(process_batch, batch))
#         for result in results:
#             encoded_chunks.extend(result)

#     vectorstore = FAISS.from_embeddings(embeddings=encoded_chunks, batch_size=batch)
#     return vectorstore

def get_vectorstore(text_chunks):
    # embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

# def get_vectorstore_parallel(text_chunks, executor):
#     embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

#     # Define the function to process a chunk
#     def process_chunk(chunk):
#         # Assuming there's a different method to obtain embeddings from the embeddings object
#         # Replace this line with the appropriate method
#         embeddings_result = embeddings.get_embeddings(chunk)
#         return embeddings_result

#     batch_size = 4
#     encoded_chunks = []
#     for i in range(0, len(text_chunks), batch_size):
#         batch = text_chunks[i:i + batch_size]
#         results = list(executor.map(process_chunk, batch))
#         encoded_chunks.extend(results)

#     # Assuming there's a different way to create a vector store from embeddings
#     # Replace this line with the appropriate method
#     vectorstore = create_vectorstore_from_embeddings(encoded_chunks)

#     return vectorstore



def get_conversation_chain(vectorstore):
    # model_path="llama-2-13b-chat.ggmlv3.q5_1.bin" # https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML
    llm_model_path = "/content/llama-2-13b-chat.ggmlv3.q5_1.bin"  # Replace with your LLM model file path

    n_gpu_layers = 40
    # n_gpu_layers = 20

    n_batch = 256
    # n_batch = 156

    callback_handler = StreamingStdOutCallbackHandler()  # Initialize the specific callback handler
    callback_manager = CallbackManager(handlers=[callback_handler])

    # Loading model,
    llm = LlamaCpp(
        model_path=llm_model_path,
        max_tokens=256,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        callback_manager=callback_manager,
        n_ctx=2024,
        verbose=False,
    )

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

def handle_userinput(user_question, conversation_chain):
    # Replace st.session_state with an appropriate data structure to hold the chat history
    chat_history = []

    response = conversation_chain({'question': user_question})
    chat_history.extend(response['chat_history'])

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(message.content)
        else:
            print(message.content)



In [ ]:
pdf_file_path = "/content/RDE_progresses.pdf"

raw_text = get_pdf_text([pdf_file_path])  # Pass the PDF file path as a list

# Rest of your code


In [ ]:
text_chunks = get_text_chunks(raw_text)

In [ ]:
# text_chunks

In [ ]:
# text_chunks = [chunk.to(device) for chunk in text_chunks]

# text_chunks = []

# num_workers = 4

# with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
#     # vectorstore = get_vectorstore_parallel(text_chunks, executor)
#     text_chunks_gpu = [chunk.to(device) for chunk in text_chunks]
#     vectorstore = get_vectorstore(text_chunks)
    # vectorstore = vectorstore.to(device)



vectorstore = get_vectorstore(get_text_chunks)
conversation_chain = get_conversation_chain(vectorstore)

load INSTRUCTOR_Transformer
max_seq_length  512


TypeError: ignored

In [ ]:

# # Example usage
# # pdf_files = ["file1.pdf", "file2.pdf"]  # Replace with your PDF filenames
# # pdf_files = ["/content/file1.pdf", "/content/file2.pdf"]  # Replace with your PDF filenames
# pdf_files ="/RDE_progresses.pdf"

# raw_text = get_pdf_text(pdf_files)
# text_chunks = get_text_chunks(raw_text)

# num_workers = 4
# with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
#     vectorstore = get_vectorstore_parallel(text_chunks, executor)

# conversation_chain = get_conversation_chain(vectorstore)

# # Example user input
# user_question = "What is this document about?"
# handle_userinput(user_question, conversation_chain)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text, chunk_size=256):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


def get_vectorstore_parallel(text_chunks, executor):
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

    # Define the function to process a chunk
    def process_chunk(chunk):
        # Assuming there's a different method to obtain embeddings from the embeddings object
        # Replace this line with the appropriate method
        embeddings_result = embeddings.get_embeddings(chunk)
        return embeddings_result

    batch_size = 4
    encoded_chunks = []
    for i in range(0, len(text_chunks), batch_size):
        batch = text_chunks[i:i + batch_size]
        results = list(executor.map(process_chunk, batch))
        encoded_chunks.extend(results)

    vectorstore = create_vectorstore_from_embeddings(encoded_chunks)

    return vectorstore



def get_conversation_chain(vectorstore):
    # model_path="llama-2-13b-chat.ggmlv3.q5_1.bin" # https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML
    llm_model_path = "/content/llama-2-13b-chat.ggmlv3.q5_1.bin"  # Replace with your LLM model file path

    n_gpu_layers = 40
    # n_gpu_layers = 20

    n_batch = 256
    # n_batch = 156

    callback_handler = StreamingStdOutCallbackHandler()  # Initialize the specific callback handler
    callback_manager = CallbackManager(handlers=[callback_handler])

    # Loading model,
    llm = LlamaCpp(
        model_path=llm_model_path,
        max_tokens=256,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        callback_manager=callback_manager,
        n_ctx=2024,
        verbose=False,
    )

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

def handle_userinput(user_question, conversation_chain):
    # Replace st.session_state with an appropriate data structure to hold the chat history
    chat_history = []

    response = conversation_chain({'question': user_question})
    chat_history.extend(response['chat_history'])

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(message.content)
        else:
            print(message.content)




In [ ]:
text_chunks = [chunk.to(device) for chunk in text_chunks]


In [ ]:
text_chunks = []

num_workers = 4

with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    vectorstore = get_vectorstore_parallel(text_chunks, executor)
    # text_chunks_gpu = [chunk.to(device) for chunk in text_chunks]
    vectorstore = get_vectorstore(text_chunks)
    # vectorstore = vectorstore.to(device)

In [ ]:

conversation_chain = get_conversation_chain(vectorstore)

# Example user input
user_question = "What is this document about?"
handle_userinput(user_question, conversation_chain)

In [ ]:
import concurrent.futures
import torch
from transformers import T5Tokenizer, BertModel, BertTokenizer
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from PyPDF2 import PdfReader

# Load the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text, chunk_size=256):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


# def get_vectorstore_parallel(text_chunks, executor):
#     # Initialize HuggingFaceInstructEmbeddings
#     embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

#     batch_size = 4
#     encoded_chunks = []
#     for i in range(0, len(text_chunks), batch_size):
#         batch = text_chunks[i:i + batch_size]
#         results = list(executor.map(embeddings.encode, batch))
#         for result in results:
#             encoded_chunks.extend(result)

#     vectorstore = FAISS.from_embeddings(embeddings=encoded_chunks, batch_size=batch)
#     return vectorstore



# def get_vectorstore_parallel(text_chunks, executor):
#     # Initialize HuggingFace tokenizer and model
#     model_name = "bert-base-uncased"
#     tokenizer = BertTokenizer.from_pretrained(model_name)
#     model = BertModel.from_pretrained(model_name)
#     model.to(device)

#     batch_size = 4
#     encoded_chunks = []
#     for i in range(0, len(text_chunks), batch_size):
#         batch = text_chunks[i:i + batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
#         with torch.no_grad():
#             inputs.to(device)
#             outputs = model(**inputs)
#             embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
#         encoded_chunks.extend(embedding)

#     vectorstore = FAISS.from_embeddings(embeddings=encoded_chunks, batch_size=batch)
#     return vectorstore





# def get_vectorstore_parallel(text_chunks, executor):
#     # Initialize HuggingFace tokenizer and model
#     model_name = "bert-base-uncased"
#     tokenizer = BertTokenizer.from_pretrained(model_name)
#     model = BertModel.from_pretrained(model_name)
#     model.to(device)

#     batch_size = 4
#     text_embeddings = []  # Collect individual text embeddings
#     for i in range(0, len(text_chunks), batch_size):
#         batch = text_chunks[i:i + batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
#         with torch.no_grad():
#             inputs.to(device)
#             outputs = model(**inputs)
#             embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
#             text_embeddings.extend(embedding)

#     # Create FAISS vector store from embeddings
#     vectorstore = FAISS.from_embeddings(text_embeddings, batch_size=batch)
#     return vectorstore



# def get_vectorstore_parallel(text_chunks, executor):
#     # Initialize HuggingFace tokenizer and model
#     model_name = "bert-base-uncased"
#     tokenizer = BertTokenizer.from_pretrained(model_name)
#     model = BertModel.from_pretrained(model_name)
#     model.to(device)

#     batch_size = 4
#     text_embeddings = []  # Collect individual text embeddings
#     for i in range(0, len(text_chunks), batch_size):
#         batch = text_chunks[i:i + batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
#         with torch.no_grad():
#             inputs.to(device)
#             outputs = model(**inputs)
#             embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
#             text_embeddings.append(embedding)

#     # Concatenate the embeddings to create a single tensor
#     text_embeddings = torch.tensor(text_embeddings).to(device)

#     # Create FAISS vector store from embeddings
#     vectorstore = FAISS.from_embeddings(embeddings=text_embeddings)
#     return vectorstore




# def get_vectorstore_parallel(text_chunks, executor):
#     # Initialize HuggingFace tokenizer and model
#     model_name = "bert-base-uncased"
#     tokenizer = BertTokenizer.from_pretrained(model_name)
#     model = BertModel.from_pretrained(model_name)
#     model.to(device)

#     batch_size = 4
#     text_embeddings = []  # Collect individual text embeddings
#     for i in range(0, len(text_chunks), batch_size):
#         batch = text_chunks[i:i + batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
#         with torch.no_grad():
#             inputs.to(device)
#             outputs = model(**inputs)
#             embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
#             text_embeddings.extend(embedding)  # Extend the list of embeddings

#     # Convert the list of embeddings to a tensor
#     text_embeddings = torch.tensor(text_embeddings).to(device)

#     # Create FAISS vector store from embeddings
#     vectorstore = FAISS.from_embeddings(embeddings=text_embeddings)
#     return vectorstore



# def get_vectorstore_parallel(text_chunks, executor):
#     # Initialize HuggingFace tokenizer and model
#     model_name = "bert-base-uncased"
#     tokenizer = BertTokenizer.from_pretrained(model_name)
#     model = BertModel.from_pretrained(model_name)
#     model.to(device)

#     batch_size = 4
#     text_embeddings = []  # Collect individual text embeddings
#     for i in range(0, len(text_chunks), batch_size):
#         batch = text_chunks[i:i + batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
#         with torch.no_grad():
#             inputs.to(device)
#             outputs = model(**inputs)
#             embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
#             text_embeddings.extend(embedding)  # Extend the list of embeddings

#     # Convert the list of embeddings to a tensor
#     text_embeddings = torch.tensor(text_embeddings).to(device)

#     # Convert embeddings to NumPy array for FAISS
#     text_embeddings_np = text_embeddings.cpu().numpy()

#     # Create FAISS vector store from embeddings
#     vectorstore = FAISS.from_embeddings(text_embeddings_np)
#     return vectorstore




def get_vectorstore_parallel(text_chunks, executor):
    # Initialize HuggingFace tokenizer and model
    model_name = "bert-base-uncased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    model.to(device)

    batch_size = 4
    text_embeddings = []  # Collect individual text embeddings
    for i in range(0, len(text_chunks), batch_size):
        batch = text_chunks[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            inputs.to(device)
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
            text_embeddings.extend(embedding)  # Extend the list of embeddings

    # Convert the list of embeddings to a tensor
    # text_embeddings = torch.tensor(text_embeddings).to(device)

    # Convert embeddings to NumPy array for FAISS
    text_embeddings_np = text_embeddings.numpy()

    # Create FAISS vector store from embeddings
    vectorstore = FAISS.from_embeddings(embeddings=text_embeddings_np, batch_size=batch_size)
    return vectorstore



# def get_vectorstore_parallel(text_chunks, executor):
#     # Initialize HuggingFace tokenizer and model
#     model_name = "bert-base-uncased"
#     tokenizer = BertTokenizer.from_pretrained(model_name)
#     model = BertModel.from_pretrained(model_name)
#     model.to(device)

#     batch_size = 4
#     text_embeddings = []  # Collect individual text embeddings
#     for i in range(0, len(text_chunks), batch_size):
#         batch = text_chunks[i:i + batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
#         with torch.no_grad():
#             inputs.to(device)
#             outputs = model(**inputs)
#             embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
#             text_embeddings.extend(embedding)

#     # Create FAISS vector store from embeddings
#     vectorstore = FAISS.from_embeddings(text_embeddings)
#     return vectorstore



def get_conversation_chain(vectorstore):
    llm_model_path = "/content/llama-2-13b-chat.ggmlv3.q5_1.bin"  # Replace with your LLM model file path
    n_gpu_layers = 40
    n_batch = 256

    callback_handler = StreamingStdOutCallbackHandler()
    callback_manager = CallbackManager(handlers=[callback_handler])

    llm = LlamaCpp(
        model_path=llm_model_path,
        max_tokens=256,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        callback_manager=callback_manager,
        n_ctx=2024,
        verbose=False,
    )

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

# Load PDF files and extract text
pdf_files =["/content/RDE_progresses.pdf"]  # Replace with your PDF filenames
raw_text = get_pdf_text(pdf_files)
text_chunks = get_text_chunks(raw_text)

# Number of workers for parallel processing
num_workers = 4

with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    # text_chunks_gpu = [chunk.to(device) for chunk in text_chunks]
    # vectorstore = get_vectorstore_parallel(text_chunks_gpu, executor)
    vectorstore = get_vectorstore_parallel(text_chunks, executor)

    vectorstore = vectorstore.to(device)

conversation_chain = get_conversation_chain(vectorstore)


AttributeError: ignored

In [ ]:
!pip install PyPDF2

In [ ]:
!pip install transformers


In [ ]:

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# simple

import concurrent.futures
# import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from transformers import AutoModel
from langchain.llms import HuggingFaceHub

# from htmlTemplates import css, bot_template, user_template



def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


def get_text_chunks(text, chunk_size=1000):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


def get_vectorstore(text_chunks):
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


def get_conversation_chain(vectorstore):
    # model_path="llama-2-13b-chat.ggmlv3.q5_1.bin" # https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML
    # model_name = "TheBloke/Llama-2-13B-chat-GGML"  # Hugging Face model name
    # model = HuggingFaceHub(repo_id="google/flan-t5-xxl")
    # from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

    # tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")
    # model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xxl")


    model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
    model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

    from huggingface_hub import hf_hub_download

    model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

    n_gpu_layers = 40
    # n_gpu_layers = 20

    n_batch = 256
    # n_batch = 156

    callback_handler = StreamingStdOutCallbackHandler()  # Initialize the specific callback handler
    callback_manager = CallbackManager(handlers=[callback_handler])

    # Loading model,
    llm = LlamaCpp(
        model_path=model_path,
        max_tokens=256,
        temperature=0.5,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        callback_manager=callback_manager,
        n_ctx=2024,
        verbose=False,
    )

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain



def handle_userinput(user_question, conversation_chain):
    # Replace st.session_state with an appropriate data structure to hold the chat history
    chat_history = []

    response = conversation_chain({'question': user_question})
    chat_history.extend(response['chat_history'])

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(message.content)
        else:
            print(message.content)


# Load PDF files and extract text
pdf_files =["/content/RDE_progresses.pdf"]  # Replace with your PDF filenames
raw_text = get_pdf_text(pdf_files)
text_chunks = get_text_chunks(raw_text)

# create vector store
vectorstore = get_vectorstore(text_chunks)
conversation_chain = get_conversation_chain(vectorstore)

# Example user input
user_question = "What is this document about?"
handle_userinput(user_question, conversation_chain)

load INSTRUCTOR_Transformer
max_seq_length  512
 This document discusses pulse detonation engines (PDEs) and their research from 2007 to 2014, including experimental work and novel concepts for space applications. It also compares different engine cycles and discusses the challenges of par

 This document discusses pulse detonation engines (PDEs) and their research from 2007 to 2014, including experimental work and novel concepts for space applications. It also compares different engine cycles and discusses the challenges of par...


# **fine tune the** embiddings, store, llm



In [1]:
!pip install streamlit langchain PyPDF2 sentence_transformers InstructorEmbedding faiss-gpu faiss-cpu llama-cpp-python transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:

import concurrent.futures
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from transformers import AutoModel
from langchain.llms import HuggingFaceHub
from huggingface_hub import hf_hub_download


def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


def get_text_chunks(text, chunk_size=1000):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


def get_vectorstore(text_chunks):
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


def get_conversation_chain(vectorstore):

    model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
    model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

    model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

    n_gpu_layers = 40
    n_batch = 256

    callback_handler = StreamingStdOutCallbackHandler()  # Initialize the specific callback handler
    callback_manager = CallbackManager(handlers=[callback_handler])

    # Loading model,
    llm = LlamaCpp(
        model_path=model_path,
        max_tokens=256,
        temperature=0.5,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        callback_manager=callback_manager,
        n_ctx=2024,
        verbose=False,
    )

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain



def handle_userinput(user_question, conversation_chain):
    # Replace st.session_state with an appropriate data structure to hold the chat history
    chat_history = []

    response = conversation_chain({'question': user_question})
    chat_history.extend(response['chat_history'])

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(message.content)
        else:
            print(message.content)



# Load PDF files and extract text
pdf_files =["/content/RDE_progresses.pdf"]  # Replace with your PDF filenames
raw_text = get_pdf_text(pdf_files)
text_chunks = get_text_chunks(raw_text)

# create vector store
vectorstore = get_vectorstore(text_chunks)
conversation_chain = get_conversation_chain(vectorstore)

# Example user input
user_question = "What is this document about?"
handle_userinput(user_question, conversation_chain)

**streamlit appy exp**

In [ ]:
!pip install streamlit langchain PyPDF2 sentence_transformers InstructorEmbedding faiss-gpu faiss-cpu llama-cpp-python transformers

In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from /content/htmlTemplates.py import css, bot_template, user_template
import concurrent.futures
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from transformers import AutoModel
from langchain.llms import HuggingFaceHub

def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


def get_vectorstore(text_chunks):
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


def get_conversation_chain(vectorstore):
    # llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})
    # model_path="llama-2-13b-chat.ggmlv3.q5_1.bin" # https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML
    # model_name = "TheBloke/Llama-2-13B-chat-GGML"  # Hugging Face model name

    model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
    model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

    model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)


    n_gpu_layers = 40
    # n_gpu_layers = 20

    n_batch = 256
    # n_batch = 156

    callback_handler = StreamingStdOutCallbackHandler()  # Initialize the specific callback handler
    callback_manager = CallbackManager(handlers=[callback_handler])

    # Loading model,
    llm = LlamaCpp(
        model_path=model_path,
        max_tokens=256,
        temperature=0.5,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        callback_manager=callback_manager,
        n_ctx=2024,
        verbose=False,
    )

    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=model_path,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain


def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.write(user_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
        else:
            st.write(bot_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)


def main():
    # load_dotenv()
    st.set_page_config(page_title="Chat with multiple PDFs",
                       page_icon=":books:")
    st.write(css, unsafe_allow_html=True)

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = None

    st.header("Chat with multiple PDFs :books:")
    user_question = st.text_input("Ask a question about your documents:")
    if user_question:
        handle_userinput(user_question)

    with st.sidebar:
        st.subheader("Your documents")
        pdf_docs = st.file_uploader(
            "Upload your PDFs here and click on 'Process'", accept_multiple_files=True)
        if st.button("Process"):
            with st.spinner("Processing"):
                # get pdf text
                raw_text = get_pdf_text(pdf_docs)

                # get the text chunks
                text_chunks = get_text_chunks(raw_text)

                # create vector store
                vectorstore = get_vectorstore(text_chunks)

                # create conversation chain
                st.session_state.conversation = get_conversation_chain(
                    vectorstore)


if __name__ == '__main__':
    main()
